タイムスタンプ

1. 【Q1】What is your age ?
2. 【Q2】What gender do you identify as ?
3. 【Q3】Which geographical region are you currently working or attending school/university in ?
4. 【Q4】Which geographical region do you most strongly associate with ?
5. 【Q5】What is your job title ?
6. 【Q6】Which group do you belong to ?
7. 【Q7】What is your research type ?
8. 【Q8】How long have you been in this field ?
9. 【Q9】Are you satisfied with your career to date ?
10. 【Q10】How many hours, on average, do you spend on housework, childcare, and caregiving per day ?
11. 【Q11】Did you already sign up for the diversity session in ICRC2023?
12. 【Q12】What do you think about the initiatives on DE&I of your group? [Gender balance]
13. 【Q12】What do you think about the initiatives on DE&I of your group? [Diversity]
14. 【Q12】What do you think about the initiatives on DE&I of your group? [Equity]
15. 【Q12】What do you think about the initiatives on DE&I of your group? [Inclusion]
16. 【Q13】What is the percentage of female researcher in your group?
17. 【Q14】What do you think about the percentage above ?
18. 【Q15】Please let us know If your group has any good practice examples related to DE&I ?
19. 【Q16】Please let us know if there is anything your group needs to work on or if your group has any problems related to DE&I.
20. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Gender balance]
21. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Diversity]
22. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Equity]
23. 【Q17】What are your thoughts on diversity, equity & inclusion initiatives ? [Inclusion]
24. 【Q18】Could you tell us more about your thoughts (agree / disagree) ?
25. 【Q19】When did you first become interested in science ?
26. 【Q20】Do you have any concerns / problems related to DE&I initiatives in science ?
27. 【Q21】What reasons do you think are hindering DE&I initiatives in science ?
28. 【Q22】Comments

In [23]:
from pathlib import Path
import pandas as pd
import altair as alt

事前アンケートの結果を読み込む

In [24]:
fname = Path("../data/test_data/20230710_icrc2023_diversity_presurvey_answers.csv")

In [113]:
data = pd.read_csv(fname, skiprows=1, parse_dates=["timestamp"])
data["response"] = 1

In [164]:
# data.head()

In [109]:
data.columns

Index(['timestamp', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9',
       'q10', 'q11', 'q12gender', 'q12diversity', 'q12equity', 'q12inclusion',
       'q13', 'q14', 'q15', 'q16', 'q17gender', 'q17diversity', 'q17equity',
       'q17inclusion', 'q18', 'q19', 'q20', 'q21', 'q22', 'response'],
      dtype='object')

In [167]:
data.index = data["timestamp"]
data_time = data.resample("H")["response"].sum().reset_index()
# data_time["yyyymmdd"] = data_time["timestamp"].dt.date
data_time["HH"] = data_time["timestamp"].dt.hour
data_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  66 non-null     datetime64[ns]
 1   response   66 non-null     int64         
 2   HH         66 non-null     int32         
dtypes: datetime64[ns](1), int32(1), int64(1)
memory usage: 1.4 KB


In [172]:
data_time

,timestamp,response,HH
0,2023-07-08 08:00:00,4,8
1,2023-07-08 09:00:00,15,9
2,2023-07-08 10:00:00,6,10
3,2023-07-08 11:00:00,5,11
4,2023-07-08 12:00:00,3,12
...,...,...,...
61,2023-07-10 21:00:00,6,21
62,2023-07-10 22:00:00,4,22
63,2023-07-10 23:00:00,6,23
64,2023-07-11 00:00:00,1,0


回答

In [178]:
alt.Chart(data_time).mark_rect().encode(
    alt.Y("date(timestamp):T").title("記入日"),
    alt.X("HH:O").title("時刻"),
    alt.Color("response"),
).properties(
    width=800,
    height=400,
).save("trend.png")

すべての質問に対してヒストグラムを作成する

In [135]:
def make_histogram(data: pd.DataFrame, x: str):
    grouped = data.groupby(x)["response"].sum().reset_index().sort_values(by=x)

    chart = alt.Chart(grouped).mark_bar().encode(
        alt.X(x),
        alt.Y("response"),
    ).properties(
        width=500,
        height=500,
    )
    return {"data": grouped, "chart": chart}

In [155]:
q = make_histogram(data, "q22")
q["data"]
q["chart"].save("chart.png")

In [157]:
for c in data.columns[1:-1]:    
    hist = make_histogram(data, c)
    fname = f"../data/quick_summary/{c}.csv"
    hist["data"].to_csv(fname, index=False)
    fname = f"../data/quick_summary/{c}.png"
    hist["chart"].save(fname)    
    

In [151]:
for _q in q:
    _q["chart"].display()
    

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

...

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [65]:
alt.Chart(data_q2).mark_bar().encode(
    alt.X("q1").title("Q1:Age"),
    alt.Y("q2").title("Q2:Gender"),
    alt.Color("response")
).properties(
    width=500,
    height=500,
)

alt.Chart(...)

In [50]:
data_q3 = data.groupby(["q3", "q4"])["response"].sum().reset_index().sort_values(by="q3")
data_q3

,q3,q4,response
0,Africa / Southern Africa,Africa / Southern Africa,2
1,America / Central America,America / Central America,1
2,America / Central America,Asia / Eastern Asia,1
3,America / North America,America / North America,19
4,America / North America,America / South America,2
5,America / North America,Asia / Central Asia,1
6,America / North America,Asia / Eastern Asia,1
7,America / North America,Asia / Southern Asia,1
8,America / North America,Asia / Western Asia / Middle East,1
9,America / North America,Europe / Central Europe,2


In [64]:
alt.Chart(data_q3).mark_bar().encode(
    alt.X("q3").title("Q3:Workplace"),
    alt.Y("q4").title("Q4:Birthplace"),
    alt.Color("response"),
).properties(
    width=500,
    height=500,
)

alt.Chart(...)